# Many particles
Next well be moving through the code to see how to create a `parcel` object that will be used to track multiple particles. First we will need to import the necessary modules.

In [1]:
import os
os.chdir('..') # move up two levels for local testing, otherwise comment out if you have installed the package
os.chdir('..')

from particula import u
from particula.aerosol_dynamics import environment, particle, parcel
import numpy as np

Now we can start creating particles and adding them to the `parcel` in a couple of different ways. 
First, we must set the `environment` and create the `parcel`. 

In [2]:
standard_environment = environment.Environment(temperature=298, pressure=101325,) # [K], [Pa] default units

simple_parcel = parcel.Parcel('simple', standard_environment)
help(simple_parcel)

Help on Parcel in module particula.aerosol_dynamics.parcel object:

class Parcel(builtins.object)
 |  Parcel(name: str, parcel_environment)
 |  
 |  Creates an air parcel to track and interact with a list of particle
 |  objects. This is the basis for particle and gas dynamic simulations.
 |  For example, evolution of a size distribution with time.
 |  
 |  Attributes:
 |      name            (str)               The name of the parcel.
 |      particle_data   (particle list)     Particle objects.
 |      environment     (object)            Parcel's environment.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name: str, parcel_environment)
 |      Constructs the parcel object.
 |      
 |      Parameters:
 |          name           (str),
 |          particle_data  (object numpy array),
 |          environment     (object).
 |  
 |  add_particle(self, particle_object)
 |      Adds a particle to the parcel.
 |      Parameters:
 |          particle_object (object)
 |  
 |  create_a

With the parcel made, we can add particles to it. First using the `add_particle` method, which is the slow way but is easy to understand.

In [3]:
#create a small and large particle
small_particle = particle.Particle(name="small_particle", radius=50e-9, density=1000, charge=0) # remember the default units are [m], [kg/m^3], [dimensionless]
large_particle = particle.Particle(name="large_particle", radius=5e-6, density=1800, charge=0)

# add single particle to parcel
simple_parcel.add_particle(small_particle)
simple_parcel.add_particle(large_particle)

Let us now look at the descriptors for the particles. Similar to the particle methods, but now we get an array in return.

In [4]:
print('radii of particles in the parcel:', simple_parcel.particle_radii_list())
print('density of particles in the parcel:', simple_parcel.particle_densities_list())
print('masses of particles in the parcel:', simple_parcel.particle_masses_list())
print('charges of particles in the parcel:', simple_parcel.particle_charges_list())

radii of particles in the parcel: [<Quantity(5e-08, 'meter')>, <Quantity(5e-06, 'meter')>]
density of particles in the parcel: [<Quantity(1000, 'kilogram / meter ** 3')>, <Quantity(1800, 'kilogram / meter ** 3')>]
masses of particles in the parcel: [<Quantity(5.23598776e-19, 'kilogram')>, <Quantity(9.42477796e-13, 'kilogram')>]
charges of particles in the parcel: [<Quantity(0, 'dimensionless')>, <Quantity(0, 'dimensionless')>]


We can also calculate the particle coagulation properties.

In [5]:
coagulation = simple_parcel.particle_dimensioned_coagulation_kernel_list(small_particle, authors='cg2019')
print('The self coagulation kernel for the small particle is:', coagulation[0])
print('The coagulation kernel for small to large particle is:', coagulation[1])

The self coagulation kernel for the small particle is: 1.4924401092516154e-15 meter ** 3 / second
The coagulation kernel for small to large particle is: 4.384030426986368e-14 meter ** 3 / second


## But what about more particles
Adding one particle at a time is good for customization, but tedious to do. To get around that we can use the `create_and_add_list_of_particle` method. This method takes an array of particle radii and creates a particle for each one.

In [6]:
# create a new parcel
muco_particle_parcel = parcel.Parcel('many_particles', standard_environment)

# create an array of radii
radii_array = np.linspace(0.1e-9, 10e-6, num=100)

# use the create_and_add_list_of_particle method to add many particles to the parcel 
muco_particle_parcel.create_and_add_list_of_particle('p_list', radii_array) # default units are [m]

You may have notice we didn't assign a density or charge to the particles, lets see what happened.

In [7]:
# just looking at the first three entries for simplicity
print('radii of particles in the parcel:', muco_particle_parcel.particle_radii_list()[0:3])
print('density of particles in the parcel:', muco_particle_parcel.particle_densities_list()[0:3])
print('charges of particles in the parcel:', muco_particle_parcel.particle_charges_list()[0:3])

radii of particles in the parcel: [<Quantity(1e-10, 'meter')>, <Quantity(1.01109091e-07, 'meter')>, <Quantity(2.02118182e-07, 'meter')>]
density of particles in the parcel: [<Quantity(1000.0, 'kilogram / meter ** 3')>, <Quantity(1000.0, 'kilogram / meter ** 3')>, <Quantity(1000.0, 'kilogram / meter ** 3')>]
charges of particles in the parcel: [<Quantity(0, 'dimensionless')>, <Quantity(0, 'dimensionless')>, <Quantity(0, 'dimensionless')>]


We can see that the particles are all created with the default density and charge, which is unit density (1000 kg/m^3) and zero charge (i.e., neutral). But lets say you don't want that to happen, and you want the particles to have different densities and charges.

We can do that by passing in an array of densities and charges to the `create_and_add_list_of_particle` method.

In [8]:
# create a new parcel
custom_particles_parcel = parcel.Parcel('many_particles', standard_environment)

# create an array of radii
radii_array = np.linspace(0.1e-9, 10e-6, num=100)
densities_array = np.linspace(1000, 1800, num=100)
charges_array = np.ones(100)

# adding many particles to the parcel, with specific radii, densities, and charges
custom_particles_parcel.create_and_add_list_of_particle('p_list', radii_array, densities_of_particles=densities_array, charges_of_particles=charges_array) # default units are [m], [kg/m^3], [dimensionless]

# let's look at the first three entries here too
print('radii of particles in the parcel:', custom_particles_parcel.particle_radii_list()[0:3])
print('density of particles in the parcel:', custom_particles_parcel.particle_densities_list()[0:3])
print('charges of particles in the parcel:', custom_particles_parcel.particle_charges_list()[0:3])

radii of particles in the parcel: [<Quantity(1e-10, 'meter')>, <Quantity(1.01109091e-07, 'meter')>, <Quantity(2.02118182e-07, 'meter')>]
density of particles in the parcel: [<Quantity(1000.0, 'kilogram / meter ** 3')>, <Quantity(1008.08081, 'kilogram / meter ** 3')>, <Quantity(1016.16162, 'kilogram / meter ** 3')>]
charges of particles in the parcel: [<Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>]


What about this warning that has been popping up? ``Warning: radius has no units, assuming [m].``

This only is displayed what you add your first particle to the `parcel`. We can fix this by adding units to the radii, but you should be aware that **if one entry has a unit, all of them must have units.**

Units are added using the `u` units module. See our units chapter or the [units](https://pint.readthedocs.io/en/stable/tutorial.html) module for more information.

In [9]:
# create a new parcel
all_the_units_parcel = parcel.Parcel('many_particles', standard_environment)

# create an array of radii
radii_array = np.linspace(50, 1000, num=100) * u.nm # units assigned are nanometer
densities_array = np.linspace(1, 1.8, num=100) * u.g / u.cm**3 # units assigned are gram / cm^3
charges_array = np.ones(100) * u.dimensionless

# adding many particles to the parcel, with specific radii, densities, and charges in units of your choice. 
all_the_units_parcel.create_and_add_list_of_particle('p_list', radii_array, densities_of_particles=densities_array, charges_of_particles=charges_array) # units are converted to [m], [kg/m^3], [dimensionless]

# let's look at the first three entries here too
print('radii of particles in the parcel:', all_the_units_parcel.particle_radii_list()[0:3])
print('density of particles in the parcel:', all_the_units_parcel.particle_densities_list()[0:3])
print('charges of particles in the parcel:', all_the_units_parcel.particle_charges_list()[0:3])

radii of particles in the parcel: [<Quantity(5e-08, 'meter')>, <Quantity(5.95959596e-08, 'meter')>, <Quantity(6.91919192e-08, 'meter')>]
density of particles in the parcel: [<Quantity(1000.0, 'kilogram / meter ** 3')>, <Quantity(1008.08081, 'kilogram / meter ** 3')>, <Quantity(1016.16162, 'kilogram / meter ** 3')>]
charges of particles in the parcel: [<Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>]


Lets say we forgot to input all the units, what does that look like?

In [10]:
# create a new parcel
missing_units_parcel = parcel.Parcel('many_particles', standard_environment)

# create an array of radii
radii_array = np.linspace(50, 1000, num=100) * u.nm # units assigned are nanometer
densities_array = np.linspace(1, 18, num=100) # no units assigned
charges_array = np.ones(100) # no units assigned

# adding many particles to the parcel, with specific radii units but no units for densities, and charges.
missing_units_parcel.create_and_add_list_of_particle('p_list', radii_array, densities_of_particles=densities_array, charges_of_particles=charges_array) # units are converted to [m], [kg/m^3], [dimensionless]

# let's look at the first three entries here too
print('radii of particles in the parcel:', missing_units_parcel.particle_radii_list()[0:3])
print('density of particles in the parcel:', missing_units_parcel.particle_densities_list()[0:3])
print('charges of particles in the parcel:', missing_units_parcel.particle_charges_list()[0:3])

radii of particles in the parcel: [<Quantity(5e-08, 'meter')>, <Quantity(5.95959596e-08, 'meter')>, <Quantity(6.91919192e-08, 'meter')>]
density of particles in the parcel: [<Quantity(1.0, 'kilogram / meter ** 3')>, <Quantity(1.17171717, 'kilogram / meter ** 3')>, <Quantity(1.34343434, 'kilogram / meter ** 3')>]
charges of particles in the parcel: [<Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>]


The output looks ok, except for very low density particles due to the default units assumed. However, the trouble comes in when we try to calculate the particle coagulation properties. 

In [11]:
missing_units_parcel.particle_dimensioned_coagulation_kernel_list(small_particle, authors='cg2019')

DimensionalityError: Cannot convert from 'nanometer ** 3' ([length] ** 3) to 'kilogram' ([mass])

Here we have a new error `DimensionalityError: Cannot convert from 'nanometer ** 3' ([length] ** 3) to 'kilogram' ([mass])`.

It is still not clear why this error happens, but to prevent it we either have no units and the defaults get assigned or assign all the units. As you can see it is not a problem for `all_the_units_parcel`.

In [ ]:
all_units_coagulation = all_the_units_parcel.particle_dimensioned_coagulation_kernel_list(small_particle, authors='cg2019')
print('The coagulation kernel to the small particle is:', all_units_coagulation[0:3])

The coagulation kernel to the small particle is: [<Quantity(1.49244011e-15, 'meter ** 3 / second')>, <Quantity(1.44723648e-15, 'meter ** 3 / second')>, <Quantity(1.44668988e-15, 'meter ** 3 / second')>]


This covers the basics of the `parcel` class. Next, we'll cover how we can use the `parcel` class to simulate aerosol coagulation.